In [1]:
import datetime as datetime
import datetime 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
events = pd.read_csv("events.csv.gzip", compression='gzip')
clicks = pd.read_csv("clicks.csv.gzip", compression='gzip')
installs = pd.read_csv("installs.csv.gzip", compression='gzip')

pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Transformo a datetime la fecha y las posiciones de clicks en floats

In [3]:
installs['created'] = pd.to_datetime(installs['created'])
events['date'] = pd.to_datetime(events['date'])
clicks['created'] = pd.to_datetime(clicks['created'])

clicks['touchX_float'] = clicks['touchX'].astype('float64')
clicks['touchY_float'] = clicks['touchY'].astype('float64')

In [4]:
installs['date_seconds_level'] = installs['created'].astype('datetime64[s]')
events['date_seconds_level'] = events['date'].astype('datetime64[s]')

### Agrego diferencias entre installs de la misma aplicacion para el mismo usuario (para borrar posibles errores de la tracking platform luego)

In [5]:
installs['seconds_to_prev_install'] = installs.sort_values(['ref_hash', 'created'], ascending=True).groupby(['ref_hash', 'application_id'])['created'].diff()
installs['seconds_to_prev_install'] = installs['seconds_to_prev_install']/np.timedelta64(1, 's')

### Elimino de installs las instalaciones con menos de un segundo de diferencia para el mismo usuario y misma applicacion (las suponemos error de tracking platform) (22077 installs menos)

In [6]:
installs = installs.loc[(installs['seconds_to_prev_install'] > 1) | pd.isnull(installs['seconds_to_prev_install'])]

### Busco los installs que no se corresponden con un evento (un install deberia tener un evento en la tabla events para esa fecha). Para ellos genero un evento (774 eventos nuevos)

In [7]:
installs_with_events = pd.merge(events, installs, on=['ref_hash', 'date_seconds_level', 'application_id'])
installs_with_events['install_with_event'] = True
installs = pd.merge(installs, installs_with_events[['ref_hash', 'date_seconds_level', 'application_id', 'install_with_event']], on=['ref_hash', 'date_seconds_level', 'application_id'], how='left')
installs.head()
installs.loc[installs['install_with_event'] != True, 'install_with_event'] = False

In [8]:
#Creo el evento por cada install no correspondido (solo los installs implicitos tienen un evento correspondiente)
events = pd.concat([events, installs.loc[(installs['install_with_event'] == False) & (installs['implicit'] == True), ['ref_hash', 'created', 'application_id']]], ignore_index=True)

/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [9]:
events.loc[pd.isnull(events['date']), 'date'] = pd.to_datetime(events['created'])
events.drop(['created'], axis=1, inplace=True)

### Agrego columnas a events, clicks y a installs para poder clasificar despues

In [10]:
installs['install_day'] = installs['created'].dt.day
events['event_day'] = events['date'].dt.day
clicks['click_day'] = clicks['created'].dt.day

### Calculo la distancia de los clicks a la cruz de cierre de la impresion (basandonos en un heatmap realizado)

In [11]:
clicks['distanceToCloseButton'] = np.sqrt(abs(clicks['touchX_float'] - 0.95)**2 + abs(clicks['touchY_float'] - 0.05)**2)

# Creando el dataframe

In [12]:
df_installs = {}
df_events = {}
df_clicks = {}
devices = {}
for i in range(18, 22):
#for i in range(24, 25):
    windowStartDate = datetime.datetime(2019, 4, i)
    windowEndDate = datetime.datetime(2019, 4, i + 3)
    df_installs[i - 18] = installs.loc[(installs['created'] >= windowStartDate) & (windowEndDate >= installs['created'])]
    df_events[i - 18] = events.loc[(events['date'] >= windowStartDate) & (windowEndDate >= events['date'])]
    df_clicks[i - 18] = clicks.loc[(clicks['created'] >= windowStartDate) & (windowEndDate >= clicks['created'])]
    
    #Calculando diferencias de tiempos para installs, eventos y clicks para un mismo usuario y esa ventana
    df_installs[i - 18]['diff_to_prev_install'] = df_installs[i - 18].sort_values('created', ascending=True).groupby('ref_hash')['created'].diff()
    df_installs[i - 18]['diff_to_prev_install'] = df_installs[i - 18]['diff_to_prev_install']/np.timedelta64(1, 's')
    df_events[i - 18]['diff_to_prev_event'] = df_events[i - 18].sort_values('date', ascending=True).groupby('ref_hash')['date'].diff()
    df_events[i - 18]['diff_to_prev_event'] = df_events[i - 18]['diff_to_prev_event']/np.timedelta64(1, 's')
    df_clicks[i - 18]['diff_to_prev_click'] = df_clicks[i - 18].sort_values('created', ascending=True).groupby('ref_hash')['created'].diff()
    df_clicks[i - 18]['diff_to_prev_click'] = df_clicks[i - 18]['diff_to_prev_click']/np.timedelta64(1, 's')
    
    #Completo los NaN con diferencias al inicio de la ventana (los NaN son aquellos que no tienen un registro previo => es el primer install/event/click para el usuario en la ventana)
    df_installs[i - 18].loc[pd.isnull(df_installs[i - 18]['diff_to_prev_install']), 'diff_to_prev_install'] = (df_installs[i - 18]['created'] - windowStartDate)/np.timedelta64(1, 's')
    df_events[i - 18].loc[pd.isnull(df_events[i - 18]['diff_to_prev_event']), 'diff_to_prev_event'] = (df_events[i - 18]['date'] - windowStartDate)/np.timedelta64(1, 's')
    df_clicks[i - 18].loc[pd.isnull(df_clicks[i - 18]['diff_to_prev_click']), 'diff_to_prev_click'] = (df_clicks[i - 18]['created'] - windowStartDate)/np.timedelta64(1, 's')
                                                                                                     
    #Creo un dataframe de dispositivos con todos los dispositivos que tienen eventos para esa ventana
    devices[i - 18] = pd.DataFrame(df_events[i - 18]['ref_hash'].unique())
    
    #Agrego los segundos a la proxima conversion (desde el inicio de la ventana)
    next_window_installs = installs.loc[(installs['created'] >= datetime.datetime(2019, 4, i + 3)) & (datetime.datetime(2019, 4, i + 6) >= installs['created'])]
    devices[i - 18].columns = ['ref_hash']
    devices[i - 18] = pd.merge(devices[i - 18], next_window_installs.groupby('ref_hash')['created'].min().reset_index(), on='ref_hash', how='left')
    devices[i - 18].rename(columns = {'created':'seconds_to_conversion'}, inplace=True)
    devices[i - 18]['seconds_to_conversion'] = (devices[i - 18]['seconds_to_conversion'] - windowEndDate)/np.timedelta64(1, 's')
    
    #Completo los seconds_to_conversion de los que no convirtieron con el valor correspondiente a 3 dias en segundos
    devices[i - 18].loc[pd.isnull(devices[i - 18]['seconds_to_conversion']), 'seconds_to_conversion'] = 259200

/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

### Separo los dataframe para obtener las distintas ventanas con las cuales entrenar

## Agrego features para cada dataframe de devices

In [13]:
for i in range(0, 4):
#for i in range(6, 7):
    #Porcentaje de installs implicitos y no implicitos en el periodo + total installs en el periodo
    installs_by_implicit = pd.pivot_table(df_installs[i], index='ref_hash', columns='implicit', values='created', aggfunc='count', fill_value=0, margins=True).reset_index()
    installs_by_implicit.drop(installs_by_implicit.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    installs_by_implicit.columns = ['ref_hash', 'installs_not_implicit', 'installs_implicit', 'total_installs']
    installs_by_implicit['ref_hash'] = installs_by_implicit['ref_hash'].astype('int64')
    installs_by_implicit['installs_not_implicit_percent'] = installs_by_implicit['installs_not_implicit']/installs_by_implicit['total_installs']
    installs_by_implicit['installs_implicit_percent'] = installs_by_implicit['installs_implicit']/installs_by_implicit['total_installs']
    devices[i] = pd.merge(devices[i], installs_by_implicit[['ref_hash', 'installs_not_implicit_percent', 'installs_implicit_percent']], on='ref_hash', how='left')
    
    #Porcentaje de installs por dia en el periodo
    installs_by_day = pd.pivot_table(df_installs[i], index='ref_hash', columns='install_day', values='created', aggfunc='count', fill_value=0, margins=True).reset_index()
    installs_by_day.drop(installs_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    installs_by_day.columns = ['ref_hash', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs']
    installs_by_day['ref_hash'] = installs_by_day['ref_hash'].astype('int64')
    installs_by_day['installs_day_1_percent'] = installs_by_day['installs_day_1']/installs_by_day['total_installs']
    installs_by_day['installs_day_2_percent'] = installs_by_day['installs_day_2']/installs_by_day['total_installs']
    installs_by_day['installs_day_3_percent'] = installs_by_day['installs_day_3']/installs_by_day['total_installs']
    devices[i] = pd.merge(devices[i], installs_by_day, on='ref_hash', how='left')
    devices[i]['total_installs_normalized'] = devices[i]['total_installs']/(devices[i]['total_installs'].max())
    del installs_by_day
    
    #Porcentaje de eventos por dia en el periodo + total eventos
    events_by_day = pd.pivot_table(df_events[i], index='ref_hash', columns='event_day', values='date', aggfunc='count', fill_value=0, margins=True).reset_index()
    events_by_day.drop(events_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    events_by_day.columns = ['ref_hash', 'events_day_1', 'events_day_2', 'events_day_3', 'total_events']
    events_by_day['ref_hash'] = events_by_day['ref_hash'].astype('int64')
    events_by_day['events_day_1_percent'] = events_by_day['events_day_1']/events_by_day['total_events']
    events_by_day['events_day_2_percent'] = events_by_day['events_day_2']/events_by_day['total_events']
    events_by_day['events_day_3_percent'] = events_by_day['events_day_3']/events_by_day['total_events']
    devices[i] = pd.merge(devices[i], events_by_day, on='ref_hash', how='left')
    devices[i]['total_events_normalized'] = devices[i]['total_events']/(devices[i]['total_events'].max())
    del events_by_day
    
    #Calculo el porcentaje de eventos hechos a traves de wifi (esto puede influir en un futuro install)
    events_by_wifi = pd.pivot_table(df_events[i], index='ref_hash', columns='wifi', values='date', aggfunc='count', fill_value=0).reset_index()
    events_by_wifi['events_on_wifi_percent'] = events_by_wifi[True]/(events_by_wifi[True] + events_by_wifi[False])
    devices[i] = pd.merge(devices[i], events_by_wifi[['ref_hash', 'events_on_wifi_percent']], on='ref_hash', how='left')
    del events_by_wifi
    
    #Porcentaje de clicks por dia en el periodo
    clicks_by_day = pd.pivot_table(df_clicks[i], index='ref_hash', columns='click_day', values='created', aggfunc='count', fill_value=0, margins=True).reset_index()
    clicks_by_day.drop(clicks_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    clicks_by_day.columns = ['ref_hash', 'clicks_day_1', 'clicks_day_2', 'clicks_day_3', 'total_clicks']
    clicks_by_day['ref_hash'] = clicks_by_day['ref_hash'].astype('int64')
    clicks_by_day['clicks_day_1_percent'] = clicks_by_day['clicks_day_1']/clicks_by_day['total_clicks']
    clicks_by_day['clicks_day_2_percent'] = clicks_by_day['clicks_day_2']/clicks_by_day['total_clicks']
    clicks_by_day['clicks_day_3_percent'] = clicks_by_day['clicks_day_3']/clicks_by_day['total_clicks']
    devices[i] = pd.merge(devices[i], clicks_by_day, on='ref_hash', how='left')
    devices[i]['total_clicks_normalized'] = devices[i]['total_clicks']/(devices[i]['total_clicks'].max())
    del clicks_by_day
    
    #Trabajando con la posicion de los clicks (promedio de distancia al boton de cerrar publicidad)
    devices[i] = pd.merge(devices[i], df_clicks[i].loc[(~np.isnan(df_clicks[i]['touchX_float'])) & (~np.isnan(df_clicks[i]['touchY_float']))].groupby('ref_hash')['distanceToCloseButton'].mean().reset_index(), on='ref_hash', how='left')
    devices[i].rename(columns = {'distanceToCloseButton':'mean_distance_to_close_button'}, inplace=True)
    
    #Agregando promedios y desvios de los tiempos entre installs, events y clicks para un mismo usuario en esa ventana
    installs_grouped = df_installs[i].groupby('ref_hash').agg({'diff_to_prev_install': ['mean', 'std']}).reset_index()
    installs_grouped.columns = ['ref_hash', 'mean_diff_between_installs', 'std_diff_between_installs']
    devices[i] = pd.merge(devices[i], installs_grouped, on='ref_hash', how='left')
    #Completo el mean_diff_between_installs con 259200 (3 dias en segundos) y el std_diff_between_installs con 0 (el desvio estandar de una muestra unica es 0)
    devices[i]['mean_diff_between_installs'].fillna(259200, inplace=True)
    devices[i]['std_diff_between_installs'].fillna(0, inplace=True)
    del installs_grouped
    
    events_grouped = df_events[i].groupby('ref_hash').agg({'diff_to_prev_event': ['mean', 'std']}).reset_index()
    events_grouped.columns = ['ref_hash', 'mean_diff_between_events', 'std_diff_between_events']
    devices[i] = pd.merge(devices[i], events_grouped, on='ref_hash', how='left')
    #Completo el mean_diff_between_events con 259200 (3 dias en segundos) y el std_diff_between_events con 0 (el desvio estandar de una muestra unica es 0)
    devices[i]['mean_diff_between_events'].fillna(259200, inplace=True)
    devices[i]['std_diff_between_events'].fillna(0, inplace=True)
    del events_grouped
    
    clicks_grouped = df_clicks[i].groupby('ref_hash').agg({'diff_to_prev_click': ['mean', 'std']}).reset_index()
    clicks_grouped.columns = ['ref_hash', 'mean_diff_between_clicks', 'std_diff_between_clicks']
    devices[i] = pd.merge(devices[i], clicks_grouped, on='ref_hash', how='left')
    #Completo el mean_diff_between_clicks con 259200 (3 dias en segundos) y el std_diff_between_clicks con 0 (el desvio estandar de una muestra unica es 0)
    devices[i]['mean_diff_between_clicks'].fillna(259200, inplace=True)
    devices[i]['std_diff_between_clicks'].fillna(0, inplace=True)
    del clicks_grouped
    
    #Completo la distancia al boton de cerrar con el promedio de la columna
    devices[i]['mean_distance_to_close_button'].fillna(devices[i]['mean_distance_to_close_button'].mean(), inplace=True)
    
    #Completo los nan con 0 todas las columnas restantes
    devices[i].fillna(0, inplace=True)

In [14]:
#Libero memoria
import gc
gc.collect()

666

## Agrego informacion del dispositivo. Esto puede tardar -- Eliminados los de events porque habia pocos datos

In [15]:
def get_most_frequent_value(s):
    return np.nan if s.isnull().all() else s.value_counts().index[0]

for i in range(0, 4):
#for i in range(6, 7):
    #Agrego los valores mas frecuentes para device_model, device_brand, ref_type y device_os en events
    #info_events = df_events[i].groupby('ref_hash')
    #devices_info_events = info_events['device_model'].apply(get_most_frequent_value).reset_index()
    #print('Agregado device_model para devices{}'.format(i))
    #devices_info_events['device_brand'] = info_events['device_brand'].apply(get_most_frequent_value).reset_index()['device_brand']
    #print('Agregado device_brand para devices{}'.format(i))
    #devices_info_events['ref_type'] = info_events['ref_type'].apply(get_most_frequent_value).reset_index()['ref_type']
    #print('Agregado ref_type para devices{}'.format(i))
    #devices_info_events['device_os'] = info_events['device_os'].apply(get_most_frequent_value).reset_index()['device_os']
    #print('Agregado device_os para devices{}'.format(i))
    #devices[i] = pd.merge(devices[i], devices_info_events, on='ref_hash', how='left')
    #del info_events, devices_info_events
    
    #Agrego los valores mas frecuentes para os_minor y os_major en clicks
    info_clicks = df_clicks[i].groupby('ref_hash')
    devices_info_clicks = info_clicks['os_minor'].apply(get_most_frequent_value).reset_index()
    print('Agregado os_minor para devices{}'.format(i))
    devices_info_clicks['os_major'] = info_clicks['os_major'].apply(get_most_frequent_value).reset_index()['os_major']
    print('Agregado os_major para devices{}'.format(i))
    devices_info_clicks['ref_type'] = info_clicks['ref_type'].apply(get_most_frequent_value).reset_index()['ref_type']
    print('Agregado ref_type para devices{}'.format(i))
    devices_info_clicks['brand'] = info_clicks['brand'].apply(get_most_frequent_value).reset_index()['brand']
    print('Agregado brand para devices{}'.format(i))
    devices_info_clicks['time_to_click_mean'] = info_clicks['timeToClick'].mean().reset_index()['timeToClick']
    print('Agregado time_to_click_mean para devices{}'.format(i))
    devices[i] = pd.merge(devices[i], devices_info_clicks, on='ref_hash', how='left')
    del info_clicks, devices_info_clicks
    
    #Completo la columna time_to_click_mean con el time_to_click_mean promedio
    devices[i]['time_to_click_mean'].fillna(devices[i]['time_to_click_mean'].mean(), inplace=True)
    print('Completados los valores nulos de time_to_click para devices{}'.format(i))
    
    #Completo la columna ref_type con el ref_type mas comun
    devices[i]['ref_type'].fillna(devices[i]['ref_type'].value_counts().index[0], inplace=True)
    print('Completados los valores nulos de time_to_click para devices{}'.format(i))
    
    #Completo la columna os_minor con el os_minor mas comun
    devices[i]['os_minor'].fillna(devices[i]['os_minor'].value_counts().index[0], inplace=True)
    print('Completados los valores nulos de os_minor para devices{}'.format(i))
    
    #Completo la columna os_major con el os_major mas comun
    devices[i]['os_major'].fillna(devices[i]['os_major'].value_counts().index[0], inplace=True)
    print('Completados los valores nulos de os_major para devices{}'.format(i))
    
    #Completo la columna brand con el brand mas comun
    devices[i]['brand'].fillna(devices[i]['brand'].value_counts().index[0], inplace=True)
    print('Completados los valores nulos de brand para devices{}'.format(i))
    
    print('----------')

Agregado os_minor para devices0
Agregado os_major para devices0
Agregado ref_type para devices0
Agregado brand para devices0
Agregado time_to_click_mean para devices0
Completados los valores nulos de time_to_click para devices0
Completados los valores nulos de time_to_click para devices0
Completados los valores nulos de os_minor para devices0
Completados los valores nulos de os_major para devices0
Completados los valores nulos de brand para devices0
----------
Agregado os_minor para devices1
Agregado os_major para devices1
Agregado ref_type para devices1
Agregado brand para devices1
Agregado time_to_click_mean para devices1
Completados los valores nulos de time_to_click para devices1
Completados los valores nulos de time_to_click para devices1
Completados los valores nulos de os_minor para devices1
Completados los valores nulos de os_major para devices1
Completados los valores nulos de brand para devices1
----------
Agregado os_minor para devices2
Agregado os_major para devices2
Agrega

In [16]:
for i in range(0, 4):
#for i in range(6, 7):
    devices[i].to_csv('devices_featured{}.csv'.format(i), header=True, index=None)
    print('Generado el archivo devices_featured{}.csv'.format(i))

Generado el archivo devices_featured0.csv
Generado el archivo devices_featured1.csv
Generado el archivo devices_featured2.csv
Generado el archivo devices_featured3.csv


# Este no es el archivo de features final. Para generar el archivo final hay que correr el otro notebook que lo mergea con los datos de auctions (no me da la capacidad de procesamiento para ejecutar todo en un mismo notebook)